In [1]:
%matplotlib inline

import math
import random
from sympy.physics.quantum.dagger import Dagger
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import sympy
from sympy import *
import itertools
from IPython.display import display
init_printing() #allows nice math displays
from qiskit_ionq import IonQProvider, GPIGate, GPI2Gate, MSGate
from qiskit.providers.jobstatus import JobStatus
from qiskit import QuantumCircuit

In [2]:
provider = IonQProvider("96TUR7QYNGMBGMyNhmG09QyAC36lj9LT")

ionq_simulator_backend = provider.get_backend("ionq_simulator", gateset = "native")
ionq_qpu_backend = provider.get_backend("ionq_qpu",gateset="native")

In this notebook, I will use the QuantumMatrix class provided in this link: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/terra/qis_adv/Clifford_Group.ipynb. It offers a good way to visualize the matrix and also takes care of the phase and coefficients thus allowing more direct comparison

In [3]:
class QuantumMatrix():
    def __init__(self, m, coeff = 1):
        self.matrix = sympy.Matrix(m)
        self.coefficient = coeff
        self.canonize()
        
    def canonize(self):
        a = next((x for x in self.matrix if x != 0), None)
        if a is not None: #zero vector
            for i,j in itertools.product([0,1], [0,1]):
                self.matrix[i,j] = sympy.simplify(self.matrix[i,j] / a)
        self.coefficient = sympy.simplify(self.coefficient * a) 
            
    def __str__(self):
        coeff_string = ""
        if self.coefficient != 1:
            coeff_string = "{} * ".format(self.coefficient)
        return "{}[[{}, {}], [{}, {}]]".format(coeff_string,self.matrix[0], self.matrix[1], self.matrix[2], self.matrix[3])
    
    def __mul__(self, rhs):
        return QuantumMatrix(self.matrix * rhs.matrix, self.coefficient * rhs.coefficient)
    
    def __add__(self, rhs):
        temp_rhs_matrix = sympy.Matrix([[1,0],[0,1]])
        for i,j in itertools.product([0,1], [0,1]):
                 temp_rhs_matrix[i,j] = sympy.simplify((rhs.matrix[i,j] * self.coefficient) / rhs.coefficient)
        return QuantumMatrix(self.matrix + temp_rhs_matrix, self.coefficient * 1/sympy.sqrt(2))
    
    def __sub__(self, rhs):
        return self + QuantumMatrix(rhs.matrix, rhs.coefficient * -1)
    
    def __eq__(self, rhs):
        return (self.matrix == rhs.matrix and self.coefficient == rhs.coefficient)
    
    def equiv(self, rhs):
        return (self.matrix == rhs.matrix)
    
    def __iter__(self):
        for x in self.matrix:
             yield x

**IonQ Single-Qubit Native Gates**

In [4]:
GPI = lambda phi : QuantumMatrix( [[0,sympy.exp(-sympy.I * phi)], 
                                   [sympy.exp(sympy.I * phi),0]])


GPI2 = lambda phi : QuantumMatrix( [[1, -sympy.I * sympy.exp(-sympy.I * phi)], 
                                     [ -sympy.I * sympy.exp(sympy.I * phi),1]], 1/sympy.sqrt(2))


**First, we define S and H gates using IonQ native gates. They will be the basis which we use to construct other Clifford gates** 

Hadamard Gates and S Gate

In [5]:
H = GPI(sympy.pi) * GPI2(sympy.pi/2)
display(sympy.Matrix(H.matrix))
print('Coefficient of H: ') 
H.coefficient

⎡1  1 ⎤
⎢     ⎥
⎣1  -1⎦

Coefficient of H: 


In [6]:
S =  GPI(sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(S.matrix))
print('Coefficient of S: ') 
S.coefficient

⎡1  0⎤
⎢    ⎥
⎣0  ⅈ⎦

Coefficient of S: 


In [7]:
I = GPI(sympy.pi/4) * GPI(sympy.pi) * GPI(-sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(I.matrix))

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

In [14]:
Sdg = GPI(-sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(Sdg.matrix))
print('Coefficient of Sdg: ') 
Sdg.coefficient
display(sympy.Matrix(Sdg.matrix))

⎡1  0 ⎤
⎢     ⎥
⎣0  -ⅈ⎦

Coefficient of Sdg: 


⎡1  0 ⎤
⎢     ⎥
⎣0  -ⅈ⎦

Pauli Groups

In [15]:
X =  GPI(0)
display(sympy.Matrix(X.matrix))
print('Coefficient of X: ')  
X.coefficient

⎡0  1⎤
⎢    ⎥
⎣1  0⎦

Coefficient of X: 


In [16]:
#0.25
Y =  GPI(sympy.pi/2)
display(sympy.Matrix(Y.matrix))
print('Coefficient of Y: ') 
Y.coefficient

⎡0   1⎤
⎢     ⎥
⎣-1  0⎦

Coefficient of Y: 


In [17]:
Z =  S * S
display(sympy.Matrix(Z.matrix))
print('Coefficient of Z: ') 
Z.coefficient

⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Coefficient of Z: 


The Axis Swap Group

In [18]:
# V = Sdg H
V =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
display(sympy.Matrix(V.matrix))
print('Coefficient of V: ') 
V.coefficient

⎡1   1⎤
⎢     ⎥
⎣-ⅈ  ⅈ⎦

Coefficient of V: 


In [19]:
# W = H S
W =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)
display(sympy.Matrix(W.matrix))
print('Coefficient of W: ') 
S.coefficient

⎡1  ⅈ ⎤
⎢     ⎥
⎣1  -ⅈ⎦

Coefficient of W: 


**24 Elements in Clifford Group**

Since the Clifford group is defined as the group of unitaries that normalize the Pauli group, we can write the one-qubit elements as $C_1 = AB$, where $A \in \{ I, V, W, H, HV, HW\}$ and $ B \in \{ I, X, Y, Z\}$ 

In [20]:
Clifford_Gates = {}

In [21]:
# X   =  GPI(sympy.pi)
# Y   =  GPI(sympy.pi/2)
# Z   =  GZ(sympy.pi/2)
# I   =  GZ(0)
# H   =  GPI(sympy.pi) * GPI2(sympy.pi/2)
# S   =  GPI(sympy.pi/4) * GPI(sympy.pi)
# Sdg =  GPI(-sympy.pi/4) * GPI(sympy.pi)
# V   =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
# W   =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)

In [22]:
# When A = I

Clifford_Gates['I'] = I
print ("I=")
display(sympy.Matrix(I.matrix))

Clifford_Gates['X'] = X
print ("X=")
display(sympy.Matrix(X.matrix))

Clifford_Gates['Y'] = Y
print ("Y=")
display(sympy.Matrix(Y.matrix))

Clifford_Gates['Z'] = Z
print ("Z=")
display(sympy.Matrix(Z.matrix))

I=


⎡1  0⎤
⎢    ⎥
⎣0  1⎦

X=


⎡0  1⎤
⎢    ⎥
⎣1  0⎦

Y=


⎡0   1⎤
⎢     ⎥
⎣-1  0⎦

Z=


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

In [23]:
# When A = V

# For simplicity, I will directly use V and XYZ gates, but the transformation to H ans S gates are below:
# V = H * S * H * S
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['V'] = V
print ("V=")
display(sympy.Matrix(V.matrix))

VX = V * X
Clifford_Gates['VX'] = VX
print ("VX=")
display(sympy.Matrix(VX.matrix))

VY = V * Y
Clifford_Gates['VY'] = VY
print ("VY=")
display(sympy.Matrix(VY.matrix))

VZ = V * Z
Clifford_Gates['VZ'] = VZ
print ("VZ=")
display(sympy.Matrix(VZ.matrix))

V=


⎡1   1⎤
⎢     ⎥
⎣-ⅈ  ⅈ⎦

VX=


⎡1  1 ⎤
⎢     ⎥
⎣ⅈ  -ⅈ⎦

VY=


⎡1  -1⎤
⎢     ⎥
⎣ⅈ  ⅈ ⎦

VZ=


⎡1   -1⎤
⎢      ⎥
⎣-ⅈ  -ⅈ⎦

In [24]:
# When A = W

# For simplicity, I will directly use W and XYZ gates, but the transformation to H ans S gates are below:
# W = S * Z * H * S * Z * H
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['W'] = W
print ("W=")
display(sympy.Matrix(W.matrix))

WX = W * X
Clifford_Gates['WX'] = WX
print ("WX=")
display(sympy.Matrix(WX.matrix))

WY = W * Y
Clifford_Gates['WY'] = WY
print ("WY=")
display(sympy.Matrix(WY.matrix))

WZ = W * Z
Clifford_Gates['WZ'] = WZ
print ("WZ=")
display(sympy.Matrix(WZ.matrix))

W=


⎡1  ⅈ ⎤
⎢     ⎥
⎣1  -ⅈ⎦

WX=


⎡1   -ⅈ⎤
⎢      ⎥
⎣-1  -ⅈ⎦

WY=


⎡1   ⅈ⎤
⎢     ⎥
⎣-1  ⅈ⎦

WZ=


⎡1  -ⅈ⎤
⎢     ⎥
⎣1  ⅈ ⎦

In [25]:
# When A = H

# For simplicity, I will directly use \ XYZ gates, but the transformation to H ans S gates are below:
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

Clifford_Gates['H'] = H
print ("H=")
display(sympy.Matrix(H.matrix))

HX = H * X
Clifford_Gates['HX'] = HX
print ("HX=")
display(sympy.Matrix(HX.matrix))

HY = H * Y
Clifford_Gates['HY'] = HY
print ("HY=")
display(sympy.Matrix(HY.matrix))

HZ = H * Z
Clifford_Gates['HZ'] = HZ
print ("HZ=")
display(sympy.Matrix(HZ.matrix))

H=


⎡1  1 ⎤
⎢     ⎥
⎣1  -1⎦

HX=


⎡1   1⎤
⎢     ⎥
⎣-1  1⎦

HY=


⎡1   -1⎤
⎢      ⎥
⎣-1  -1⎦

HZ=


⎡1  -1⎤
⎢     ⎥
⎣1  1 ⎦

In [26]:
# When A = HV

# For simplicity, I will directly use HV and XYZ gates, but the transformation to H ans S gates are below:
# W = S * Z * H * S * Z * H
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

HV = H * V
Clifford_Gates['HV'] = HV
print ("HV=")
display(sympy.Matrix(HV.matrix))

HVX = HV * X
Clifford_Gates['HVX'] = HVX
print ("HVX=")
display(sympy.Matrix(HVX.matrix))

HVY = HV * Y
Clifford_Gates['HVY'] = HVY
print ("HVY=")
display(sympy.Matrix(HVY.matrix))

HVZ = HV * Z
Clifford_Gates['HVZ'] = HVZ
print ("HVZ=")
display(sympy.Matrix(HVZ.matrix))

HV=


⎡1  ⅈ⎤
⎢    ⎥
⎣ⅈ  1⎦

HVX=


⎡1   -ⅈ⎤
⎢      ⎥
⎣-ⅈ  1 ⎦

HVY=


⎡1   ⅈ ⎤
⎢      ⎥
⎣-ⅈ  -1⎦

HVZ=


⎡1  -ⅈ⎤
⎢     ⎥
⎣ⅈ  -1⎦

In [27]:
# When A = HW

# For simplicity, I will directly use W and XYZ gates, but the transformation to H ans S gates are below:
# HW = H * S * H * S
# X = H * S * S * H
# Y = S * X * Sdg
# Z = S * S

HW = H * W
Clifford_Gates['HW'] = HW
print ("HW=")
display(sympy.Matrix(HW.matrix))

HWX = HW * X
Clifford_Gates['HWX'] = HWX
print ("HWX=")
display(sympy.Matrix(HWX.matrix))

HWY = HW * Y
Clifford_Gates['HWY'] = HWY
print ("HWY=")
display(sympy.Matrix(HWY.matrix))

HWZ = HW * Z
Clifford_Gates['HWZ'] = HWZ
print ("HWZ=")
display(sympy.Matrix(HWZ.matrix))

HW=


⎡1  0⎤
⎢    ⎥
⎣0  ⅈ⎦

HWX=


⎡0  1⎤
⎢    ⎥
⎣ⅈ  0⎦

HWY=


⎡0   1⎤
⎢     ⎥
⎣-ⅈ  0⎦

HWZ=


⎡1  0 ⎤
⎢     ⎥
⎣0  -ⅈ⎦

In [28]:
len(Clifford_Gates)

In [29]:
# X   =  GPI(sympy.pi)
# Y   =  GPI(sympy.pi/2)
# Z   =  SS = GPI(sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi/4) * GPI(sympy.pi)
# I   =  SS+ = GPI(sympy.pi/4) * GPI(sympy.pi) * GPI(-sympy.pi/4) * GPI(sympy.pi)
# H   =  GPI(sympy.pi) * GPI2(sympy.pi/2)
# S   =  GPI(sympy.pi/4) * GPI(sympy.pi)
# Sdg =  GPI(-sympy.pi/4) * GPI(sympy.pi)
# V   =  GPI(-sympy.pi/4) * GPI(sympy.pi) * GPI(sympy.pi) * GPI2(sympy.pi/2)
# W   =  GPI(sympy.pi) * GPI2(sympy.pi/2) * GPI(sympy.pi/4) * GPI(sympy.pi)

In [30]:
def constructGate(qc, gate):
    if (gate == 'X'):
        qc.append(GPIGate(0.5), [0])
    elif (gate == 'Y'):
        qc.append(GPIGate(0.25), [0])
    elif (gate == 'Z'):
        qc.append(GPIGate(0), [0])
        qc.append(GPIGate(0.25), [0])
#    elif (gate == 'I'):       
    elif (gate == 'H'):
        qc.append(GPIGate(0.5), [0])
        qc.append(GPI2Gate(0.25), [0])
    elif (gate == 'V'):
        qc.append(GPIGate(-0.125), [0])
        qc.append(GPIGate(0.5), [0])
        qc.append(GPIGate(0.5), [0])
        qc.append(GPI2Gate(0.25), [0])
    elif (gate == 'W'):
        qc.append(GPIGate(0.5), [0])
        qc.append(GPI2Gate(0.25), [0])
        qc.append(GPIGate(0.125), [0])
        qc.append(GPIGate(0.5), [0])
    return qc

In [31]:
def constructCircuit(qc,L,gate_set_mat,ref_gates):
    
    for i in range(len(L)):
        clifford_gate = split(L[i])
        for j in range(len(clifford_gate)):
            qc = constructGate(qc, clifford_gate[j])
        qc.draw()
    qc = addInverseToCircuit(qc,gate_set_mat,ref_gates)
    qc.measure(0,0)
    return qc

In [32]:
def split(word):
    return [char for char in word]

In [33]:
def check_truth(list):
    for i in range(len(list)):
        if (list[i] == False):
            return False
    return True

def get_inverse_matrix(clifford_string_mat,ref_gates):
    A = QuantumMatrix( [[1,0],
                        [0,1]])
    imag_phase = QuantumMatrix([[sympy.I,0],
                                [0,sympy.I]])
    negative_phase = QuantumMatrix([[-1,0],
                                    [0,-1]])
    neg_imag_phase = QuantumMatrix([[-sympy.I,0],
                                [0,-sympy.I]])
    for i in range(len(clifford_string_mat)):
        A*=clifford_string_mat[i]
    print('Total Operation:')
    display(sympy.Matrix(A.matrix))
    A = Dagger(A.matrix)
    for items in ref_gates.items():
        truths_list = []
        imag_truths_list = []
        neg_truths_list = []
        imag_neg_truths_list = []
        for i in range(2):
            for j in range(2):
                truths_list.append((A[i,j]) == items[1].matrix[i,j])
                imag_truths_list.append((A[i,j]) == sympy.I*items[1].matrix[i,j])
                neg_truths_list.append((A[i,j]) == -1*items[1].matrix[i,j])
                imag_neg_truths_list.append((A[i,j]) == -sympy.I*items[1].matrix[i,j])
        if (check_truth(truths_list)):
            return items[0]
        elif (check_truth(imag_truths_list)):
            return items[0]
        elif (check_truth(neg_truths_list)):
            return items[0]
        elif (check_truth(imag_neg_truths_list)):
            return items[0]

In [34]:
def get_random_clifford_string(L,gates):
    entry_list = list(gates.items())
    cliff_string_name = []
    cliff_string_mat = []
    for i in range(L):
        random_choice = random.choice(entry_list)
        cliff_string_name.append(random_choice[0])
        cliff_string_mat.append(random_choice[1])
    return cliff_string_name, cliff_string_mat

In [35]:
def addInverseToCircuit(qc,gate_set_mat,Clifford_Gates):
    inverse_gate_name = get_inverse_matrix(gate_set_mat,Clifford_Gates)
    print ("Inverse Operation: ", inverse_gate_name)
    display(sympy.Matrix(Clifford_Gates[inverse_gate_name].matrix))
    
    inverse_gates = split(inverse_gate_name)
    for i in range(len(inverse_gates)):
        qc = constructGate(qc, inverse_gates[i])
    return qc

In [36]:
L = 200
gate_set_name, gate_set_mat = get_random_clifford_string(L, Clifford_Gates)

In [38]:
qc = QuantumCircuit(1,1, name=f"gate-native")


final_circuit = constructCircuit(qc,gate_set_name,gate_set_mat,Clifford_Gates)

Total Operation:


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Inverse Operation:  Z


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

In [39]:
job = ionq_simulator_backend.run(final_circuit, shots=1000) 

# Print the counts
print(job.get_counts().get("0"))

1000


In [62]:
L=20
number_shots=1000
number_runs=10
results_20 = []
qc_20 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_20, gate_set_mat_20 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_20)
final_circuit_20 = constructCircuit(qc_20,gate_set_name_20,gate_set_mat_20,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_20,shots=number_shots)
    results_20.append(job.get_counts().get("0")/number_shots)
    print(results_20)


['H', 'HW', 'HVY', 'H', 'HZ', 'HVZ', 'VY', 'HWX', 'HVX', 'VZ', 'HVZ', 'HY', 'HY', 'HX', 'HZ', 'HW', 'HW', 'HVX', 'HY', 'WZ']
Total Operation:


⎡0  1⎤
⎢    ⎥
⎣ⅈ  0⎦

Inverse Operation:  HWX


⎡0  1⎤
⎢    ⎥
⎣ⅈ  0⎦

[0.87]
[0.87, 0.867]
[0.87, 0.867, 0.865]
[0.87, 0.867, 0.865, 0.887]
[0.87, 0.867, 0.865, 0.887, 0.872]
[0.87, 0.867, 0.865, 0.887, 0.872, 0.878]
[0.87, 0.867, 0.865, 0.887, 0.872, 0.878, 0.881]
[0.87, 0.867, 0.865, 0.887, 0.872, 0.878, 0.881, 0.888]
[0.87, 0.867, 0.865, 0.887, 0.872, 0.878, 0.881, 0.888, 0.866]
[0.87, 0.867, 0.865, 0.887, 0.872, 0.878, 0.881, 0.888, 0.866, 0.876]


In [72]:
gate_set_name_20

['H',
 'HW',
 'HVY',
 'H',
 'HZ',
 'HVZ',
 'VY',
 'HWX',
 'HVX',
 'VZ',
 'HVZ',
 'HY',
 'HY',
 'HX',
 'HZ',
 'HW',
 'HW',
 'HVX',
 'HY',
 'WZ']

In [64]:
L=30
number_shots=1000
number_runs=10
results_30 = []
qc_30 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_30, gate_set_mat_30 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_30)
final_circuit_30 = constructCircuit(qc_30,gate_set_name_30,gate_set_mat_30,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_30,shots=number_shots)
    results_30.append(job.get_counts().get("0")/number_shots)
    print(results_30)



['VX', 'HWY', 'H', 'WZ', 'V', 'V', 'HWZ', 'HWX', 'HV', 'HX', 'HVY', 'W', 'HWX', 'HVY', 'HWX', 'X', 'HWY', 'HV', 'HWY', 'Z', 'HY', 'V', 'W', 'I', 'HZ', 'HWX', 'Z', 'WX', 'HV', 'HX']
Total Operation:


⎡1   ⅈ⎤
⎢     ⎥
⎣-1  ⅈ⎦

Inverse Operation:  VZ


⎡1   -1⎤
⎢      ⎥
⎣-ⅈ  -ⅈ⎦

[0.607]
[0.607, 0.545]
[0.607, 0.545, 0.515]
[0.607, 0.545, 0.515, 0.692]
[0.607, 0.545, 0.515, 0.692, 0.644]
[0.607, 0.545, 0.515, 0.692, 0.644, 0.639]
[0.607, 0.545, 0.515, 0.692, 0.644, 0.639, 0.724]
[0.607, 0.545, 0.515, 0.692, 0.644, 0.639, 0.724, 0.638]
[0.607, 0.545, 0.515, 0.692, 0.644, 0.639, 0.724, 0.638, 0.648]
[0.607, 0.545, 0.515, 0.692, 0.644, 0.639, 0.724, 0.638, 0.648, 0.561]


In [65]:
L=40
number_shots=1000
number_runs=10
results_40 = []
qc_40 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_40, gate_set_mat_40 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_40)
final_circuit_40 = constructCircuit(qc_40,gate_set_name_40,gate_set_mat_40,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_40,shots=number_shots)
    results_40.append(job.get_counts().get("0")/number_shots)
    print(results_40)



['VZ', 'HX', 'Z', 'Z', 'HY', 'HVZ', 'I', 'Z', 'HY', 'HVX', 'HW', 'I', 'W', 'X', 'I', 'HVX', 'HY', 'VZ', 'HWX', 'HW', 'Z', 'X', 'Y', 'W', 'VX', 'HX', 'HVZ', 'WX', 'HVY', 'HVY', 'HY', 'VX', 'X', 'I', 'HZ', 'HVZ', 'VX', 'HVX', 'HVZ', 'HV']
Total Operation:


⎡1  1 ⎤
⎢     ⎥
⎣ⅈ  -ⅈ⎦

Inverse Operation:  WZ


⎡1  -ⅈ⎤
⎢     ⎥
⎣1  ⅈ ⎦

[0.709]
[0.709, 0.755]
[0.709, 0.755, 0.744]
[0.709, 0.755, 0.744, 0.686]
[0.709, 0.755, 0.744, 0.686, 0.631]
[0.709, 0.755, 0.744, 0.686, 0.631, 0.763]
[0.709, 0.755, 0.744, 0.686, 0.631, 0.763, 0.617]
[0.709, 0.755, 0.744, 0.686, 0.631, 0.763, 0.617, 0.572]
[0.709, 0.755, 0.744, 0.686, 0.631, 0.763, 0.617, 0.572, 0.573]
[0.709, 0.755, 0.744, 0.686, 0.631, 0.763, 0.617, 0.572, 0.573, 0.777]


In [68]:
L=50
number_shots=1000
number_runs=10
results_50 = []
qc_50 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_50, gate_set_mat_50 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_50)
final_circuit_50 = constructCircuit(qc_50,gate_set_name_50,gate_set_mat_50,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_50,shots=number_shots)
    results_50.append(job.get_counts().get("0")/number_shots)
    print(results_50)

['X', 'HY', 'HVX', 'W', 'HV', 'WZ', 'HWX', 'VZ', 'HWZ', 'Z', 'I', 'Y', 'VY', 'VZ', 'VY', 'HW', 'V', 'WX', 'HWY', 'WZ', 'W', 'I', 'VZ', 'HWY', 'V', 'HWX', 'WZ', 'HWY', 'V', 'V', 'HWZ', 'HWY', 'HWY', 'HWX', 'HWX', 'HVX', 'X', 'HWX', 'Y', 'WX', 'HV', 'HY', 'VZ', 'Z', 'VY', 'VZ', 'HWX', 'HWX', 'HX', 'WY']
Total Operation:


⎡0  1⎤
⎢    ⎥
⎣1  0⎦

Inverse Operation:  X


⎡0  1⎤
⎢    ⎥
⎣1  0⎦

[0.888]
[0.888, 0.77]
[0.888, 0.77, 0.759]
[0.888, 0.77, 0.759, 0.77]
[0.888, 0.77, 0.759, 0.77, 0.671]
[0.888, 0.77, 0.759, 0.77, 0.671, 0.611]
[0.888, 0.77, 0.759, 0.77, 0.671, 0.611, 0.636]
[0.888, 0.77, 0.759, 0.77, 0.671, 0.611, 0.636, 0.703]
[0.888, 0.77, 0.759, 0.77, 0.671, 0.611, 0.636, 0.703, 0.869]
[0.888, 0.77, 0.759, 0.77, 0.671, 0.611, 0.636, 0.703, 0.869, 0.861]


In [69]:
L=75
number_shots=1000
number_runs=10
results_75 = []
qc_75 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_75, gate_set_mat_75 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_75)
final_circuit_75 = constructCircuit(qc_75,gate_set_name_75,gate_set_mat_75,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_75,shots=number_shots)
    results_75.append(job.get_counts().get("0")/number_shots)
    print(results_75)

['HWZ', 'HV', 'HVZ', 'I', 'V', 'VY', 'HVX', 'WX', 'V', 'W', 'WY', 'I', 'X', 'X', 'HV', 'HV', 'HVZ', 'WZ', 'VX', 'HWZ', 'VY', 'I', 'WY', 'VX', 'HX', 'VY', 'VY', 'HVZ', 'VY', 'HVY', 'WY', 'HWY', 'HV', 'Z', 'HVY', 'HV', 'HWY', 'HZ', 'H', 'VX', 'Y', 'HW', 'HX', 'HY', 'H', 'HZ', 'V', 'HW', 'Z', 'HZ', 'HY', 'HWZ', 'VY', 'HZ', 'HVX', 'HW', 'I', 'Y', 'HX', 'W', 'X', 'W', 'HWX', 'VZ', 'H', 'W', 'HY', 'HVZ', 'VY', 'HWZ', 'HVX', 'HVX', 'HWZ', 'HVZ', 'W']
Total Operation:


⎡1  -ⅈ⎤
⎢     ⎥
⎣1  ⅈ ⎦

Inverse Operation:  VX


⎡1  1 ⎤
⎢     ⎥
⎣ⅈ  -ⅈ⎦

[0.537]
[0.537, 0.499]
[0.537, 0.499, 0.567]
[0.537, 0.499, 0.567, 0.503]
[0.537, 0.499, 0.567, 0.503, 0.79]
[0.537, 0.499, 0.567, 0.503, 0.79, 0.463]
[0.537, 0.499, 0.567, 0.503, 0.79, 0.463, 0.736]
[0.537, 0.499, 0.567, 0.503, 0.79, 0.463, 0.736, 0.735]
[0.537, 0.499, 0.567, 0.503, 0.79, 0.463, 0.736, 0.735, 0.766]
[0.537, 0.499, 0.567, 0.503, 0.79, 0.463, 0.736, 0.735, 0.766, 0.523]


In [71]:
L=100
number_shots=1000
number_runs=10
results_100 = []
qc_100 = QuantumCircuit(1,1,name=f"gate-native")
gate_set_name_100, gate_set_mat_100 = get_random_clifford_string(L, Clifford_Gates)
print(gate_set_name_100)
final_circuit_100 = constructCircuit(qc_100,gate_set_name_100,gate_set_mat_100,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_100,shots=number_shots)
    results_100.append(job.get_counts().get("0")/number_shots)
    print(results_100)

['I', 'V', 'HWY', 'X', 'V', 'WY', 'H', 'VY', 'HY', 'V', 'WZ', 'HVX', 'HWX', 'H', 'I', 'HV', 'WX', 'HZ', 'HWY', 'HX', 'HWY', 'HWX', 'WY', 'HZ', 'X', 'HX', 'HWX', 'HWY', 'HV', 'VX', 'X', 'HWZ', 'I', 'HWZ', 'I', 'Y', 'HW', 'WZ', 'HVY', 'W', 'HY', 'I', 'Z', 'VZ', 'HY', 'HW', 'VZ', 'WY', 'WZ', 'WY', 'VZ', 'H', 'Z', 'WZ', 'I', 'VX', 'WY', 'HV', 'HWX', 'Z', 'HY', 'VY', 'HX', 'HY', 'HY', 'VY', 'Y', 'HWY', 'HY', 'VZ', 'HVZ', 'WX', 'VX', 'HVX', 'X', 'HVX', 'Y', 'I', 'HY', 'Y', 'W', 'HV', 'HVX', 'HWX', 'WZ', 'WZ', 'WX', 'W', 'Y', 'X', 'V', 'Z', 'HWY', 'HVY', 'HVX', 'X', 'WX', 'HZ', 'WZ', 'V']
Total Operation:


⎡1   -1⎤
⎢      ⎥
⎣-ⅈ  -ⅈ⎦

Inverse Operation:  WY


⎡1   ⅈ⎤
⎢     ⎥
⎣-1  ⅈ⎦

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.ionq.co | 524: A timeout occurred</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


<script>
(function(){if(document.addEventListener&&window.XMLHttpRequest&&JSON&&JSON.stringify){var e=function(a){var c=document.getElementById("error-feedback-survey"),d=document.getElementById("error-feedback-success"),b=new XMLH

In [80]:
def addInverseToCircuit_2qubit(qc,gate_set_mat,Clifford_Gates):
    inverse_gate_name = get_inverse_matrix(gate_set_mat,Clifford_Gates)
    print ("Inverse Operation: ", inverse_gate_name)
    display(sympy.Matrix(Clifford_Gates[inverse_gate_name].matrix))
    
    inverse_gates = split(inverse_gate_name)
    for i in range(len(inverse_gates)):
        qc = constructGate_2qubit(qc, inverse_gates[i],0)
        qc = constructGate_2qubit(qc, inverse_gates[i],1)
    return qc

In [81]:
def constructGate_2qubit(qc, gate,qubit):
    if (gate == 'X'):
        qc.append(GPIGate(0.5), [qubit])
    elif (gate == 'Y'):
        qc.append(GPIGate(0.25), [qubit])
    elif (gate == 'Z'):
        qc.append(GPIGate(0), [qubit])
        qc.append(GPIGate(0.25), [qubit])
#    elif (gate == 'I'):       
    elif (gate == 'H'):
        qc.append(GPIGate(0.5), [qubit])
        qc.append(GPI2Gate(0.25), [qubit])
    elif (gate == 'V'):
        qc.append(GPIGate(-0.125), [qubit])
        qc.append(GPIGate(0.5), [qubit])
        qc.append(GPIGate(0.5), [qubit])
        qc.append(GPI2Gate(0.25), [qubit])
    elif (gate == 'W'):
        qc.append(GPIGate(0.5), [qubit])
        qc.append(GPI2Gate(0.25), [qubit])
        qc.append(GPIGate(0.125), [qubit])
        qc.append(GPIGate(0.5), [qubit])
    return qc

In [82]:
def constructCircuit_2qubit(qc,L,gate_set_mat,ref_gates):
    
    for i in range(len(L)):
        clifford_gate = split(L[i])
        for j in range(len(clifford_gate)):
            qc = constructGate_2qubit(qc, clifford_gate[j],[0])
            qc = constructGate_2qubit(qc, clifford_gate[j],[1])


    qc = addInverseToCircuit_2qubit(qc,gate_set_mat,ref_gates)
    qc = addInverseToCircuit_2qubit(qc,gate_set_mat,ref_gates)
    
    qc.measure(0,0)
    qc.measure(1,1)
    return qc

In [ ]:
L=20
number_shots=1000
number_runs=10
results_20_2qubit = []
qc_20_2qubit = QuantumCircuit(2,2,name=f"gate-native")


print(gate_set_name_20)

final_circuit_20_2qubit = constructCircuit_2qubit(qc_20_2qubit,gate_set_name_20,gate_set_mat_20,Clifford_Gates)
for i in range(number_runs):
    job = ionq_qpu_backend.run(final_circuit_20_2qubit,shots=number_shots)
    results_20_2qubit.append(job.get_counts().get("00")/number_shots)
    print(results_20_2qubit)



In [83]:
qc = QuantumCircuit(2,2, name=f"gate-native")


final_circuit = constructCircuit_2qubit(qc,gate_set_name,gate_set_mat,Clifford_Gates)

Total Operation:


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Inverse Operation:  Z


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Total Operation:


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

Inverse Operation:  Z


⎡1  0 ⎤
⎢     ⎥
⎣0  -1⎦

In [84]:
job = ionq_simulator_backend.run(final_circuit, shots=1000) 

# Print the counts
#print(job.get_counts().get("0"))

In [85]:
job.get_counts()

{'00': 1000}